In [1]:
import mne
import glob
import numpy as np
import pandas as pd
files = glob.glob('eeg_data_task_rest/*.edf')
len(files)

72

In [2]:
rest_files = [i for i in files if i[-5] == '1']
task_files = [i for i in files if i[-5] == '2']
len(rest_files), len(task_files)

(36, 36)

In [3]:
def read_data(file_path):
    data = mne.io.read_raw_edf(file_path, preload=True)
    data.set_eeg_reference()
    data.filter(l_freq = 0.5, h_freq = 50)
    epochs = mne.make_fixed_length_epochs(data, duration =5, overlap = 1)
    array = epochs.get_data()
    return array

In [4]:
%%capture
rest_epochs_array = [read_data(i) for i in rest_files]
task_epochs_array = [read_data(i) for i in task_files]

In [5]:
rest_epochs_label = [len(i)*[0] for i in rest_epochs_array]
task_epochs_label = [len(i)*[1] for i in task_epochs_array]
data_list = rest_epochs_array+task_epochs_array
label_list = rest_epochs_label+task_epochs_label

In [6]:
data_array = np.vstack(data_list)
label_array = np.hstack(label_list)

In [7]:
data_array = np.moveaxis(data_array, 1, 2)
data_array.shape

(2132, 2500, 21)

In [8]:
from keras.models import Model
from keras.layers import Input, Conv1D, MaxPooling1D, AveragePooling1D, concatenate, Dense, Flatten

# Define the input shape
input_shape = (2500, 21)
# Define the Tception model
inputs = Input(shape=input_shape)

# Tception block 1
x1 = Conv1D(64, 3, activation='relu', padding='same')(inputs)
x1 = MaxPooling1D(2)(x1)

x2 = Conv1D(64, 5, activation='relu', padding='same')(inputs)
x2 = MaxPooling1D(2)(x2)

x3 = Conv1D(64, 7, activation='relu', padding='same')(inputs)
x3 = MaxPooling1D(2)(x3)

x = concatenate([x1, x2, x3])

# Tception block 2
x1 = Conv1D(128, 3, activation='relu', padding='same')(x)
x1 = AveragePooling1D(2)(x1)

x2 = Conv1D(128, 5, activation='relu', padding='same')(x)
x2 = AveragePooling1D(2)(x2)

x3 = Conv1D(128, 7, activation='relu', padding='same')(x)
x3 = AveragePooling1D(2)(x3)

x = concatenate([x1, x2, x3])

# Flatten and Dense layers
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)

# Define the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_array, label_array, test_size=0.3)

In [13]:
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test), batch_size=32)

Epoch 1/5
47/47 ━━━━━━━━━━━━━━━━━━━━ 29s 587ms/step - accuracy: 0.7164 - loss: 0.6260 - val_accuracy: 0.7422 - val_loss: 0.5756
Epoch 2/5
47/47 ━━━━━━━━━━━━━━━━━━━━ 27s 571ms/step - accuracy: 0.7645 - loss: 0.5553 - val_accuracy: 0.7422 - val_loss: 0.5708
Epoch 3/5
47/47 ━━━━━━━━━━━━━━━━━━━━ 28s 590ms/step - accuracy: 0.7571 - loss: 0.5576 - val_accuracy: 0.7422 - val_loss: 0.5720
Epoch 4/5
47/47 ━━━━━━━━━━━━━━━━━━━━ 28s 587ms/step - accuracy: 0.7471 - loss: 0.5677 - val_accuracy: 0.7422 - val_loss: 0.5734
Epoch 5/5
47/47 ━━━━━━━━━━━━━━━━━━━━ 29s 609ms/step - accuracy: 0.7421 - loss: 0.5737 - val_accuracy: 0.7422 - val_loss: 0.5708


F1 score can be calculated as shown in eegnet code which here will come out around to be 0.4-0.5

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(X_train.shape[0], -1)).reshape(X_train.shape)
X_test_scaled = scaler.transform(X_test.reshape(X_test.shape[0], -1)).reshape(X_test.shape)

In [11]:
model_scaled =  Model(inputs=inputs, outputs=outputs)
model_scaled.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
model_scaled.fit(X_train_scaled, y_train, epochs=5, validation_data=(X_test_scaled, y_test), batch_size=32)

Epoch 1/5
47/47 ━━━━━━━━━━━━━━━━━━━━ 48s 902ms/step - accuracy: 0.6474 - loss: 7.5962 - val_accuracy: 0.7609 - val_loss: 0.5996
Epoch 2/5
47/47 ━━━━━━━━━━━━━━━━━━━━ 41s 869ms/step - accuracy: 0.7415 - loss: 0.5888 - val_accuracy: 0.7609 - val_loss: 0.5459
Epoch 3/5
47/47 ━━━━━━━━━━━━━━━━━━━━ 41s 861ms/step - accuracy: 0.7460 - loss: 0.5573 - val_accuracy: 0.7609 - val_loss: 0.5405
Epoch 4/5
47/47 ━━━━━━━━━━━━━━━━━━━━ 41s 869ms/step - accuracy: 0.7396 - loss: 0.5627 - val_accuracy: 0.7609 - val_loss: 0.5370
Epoch 5/5
47/47 ━━━━━━━━━━━━━━━━━━━━ 41s 879ms/step - accuracy: 0.7496 - loss: 0.5288 - val_accuracy: 0.7625 - val_loss: 0.4988


We can see that val_loss is decreasing while val_acc isnt, this can be due to low amount of data